# 📊 EDA Completo - Berka Dataset (Czech Banking Data)

## 🎯 Objetivo del Análisis

Este notebook contiene el **Análisis Exploratorio de Datos (EDA) completo** sobre una muestra del 5% del Berka Dataset. Los objetivos son:

1. **Identificar problemas de calidad de datos** (nulos, duplicados, inconsistencias)
2. **Validar integridad estructural y relacional** (PKs, FKs, tipos de datos)
3. **Descubrir patrones de negocio** (comportamiento transaccional, riesgo crediticio)
4. **Definir el plan de transformación ETL** para las capas Raw → Processed → Curated

---

## 📚 Tabla de Contenidos

1. [Configuración y Carga de Datos](#1-configuracion)
2. [Análisis de Estructura y Esquema](#2-estructura)
3. [Análisis de Calidad de Datos](#3-calidad)
4. [Análisis de Integridad Relacional](#4-integridad)
5. [Análisis Descriptivo por Tabla](#5-descriptivo)
6. [Análisis de Negocio y Patrones](#6-negocio)
7. [Análisis de Outliers y Anomalías](#7-outliers)
8. [Conclusiones y Plan ETL](#8-conclusiones)



<a id='1-configuracion'></a>
## 1. ⚙️ Configuración y Carga de Datos

### 1.1 Importaciones y Configuración Inicial

In [10]:
import pandas as pd
import numpy as np
import os
import random
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# --- CONFIGURACIÓN DEL PROYECTO ---
INPUT_DIR = './data_original'
OUTPUT_DIR = './data/raw/berka'
SAMPLE_PERCENTAGE = 0.05  # Muestra del 5%
RANDOM_SEED = 42
SEPARATOR = ';'

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# Archivos del dataset
CSV_FILES = [
    'account.csv', 'client.csv', 'disp.csv', 'loan.csv', 
    'order.csv', 'trans.csv', 'district.csv', 'card.csv'
]

# Diccionario para almacenar DataFrames
dataframes = {}

# Definición de Claves Primarias
PRIMARY_KEYS = {
    'ACCOUNT': 'account_id',
    'CLIENT': 'client_id',
    'DISP': 'disp_id', 
    'LOAN': 'loan_id',
    'ORDER': 'order_id',
    'TRANS': 'trans_id',
    'DEMOGRAPHIC': 'a1',  # district_id
    'CARD': 'card_id'
}

# Definición de Claves Foráneas (Relaciones)
FOREIGN_KEYS = {
    ('DISP', 'account_id'): ('ACCOUNT', 'account_id'),
    ('DISP', 'client_id'): ('CLIENT', 'client_id'),
    ('LOAN', 'account_id'): ('ACCOUNT', 'account_id'),
    ('ORDER', 'account_id'): ('ACCOUNT', 'account_id'),
    ('TRANS', 'account_id'): ('ACCOUNT', 'account_id'),
    ('CARD', 'disp_id'): ('DISP', 'disp_id'),
    ('ACCOUNT', 'district_id'): ('DEMOGRAPHIC', 'a1')
}

print("✅ Configuración inicial completada")
print(f"📁 Directorio de entrada: {INPUT_DIR}")
print(f"📁 Directorio de salida: {OUTPUT_DIR}")
print(f"📊 Porcentaje de muestra: {SAMPLE_PERCENTAGE * 100}%")



✅ Configuración inicial completada
📁 Directorio de entrada: ./data_original
📁 Directorio de salida: ./data/raw/berka
📊 Porcentaje de muestra: 5.0%


### 1.2 Funciones Auxiliares

In [11]:
def limpiar_nombres_columnas(df):
    """
    Limpia los nombres de las columnas:
    - Elimina comillas y espacios
    - Convierte a minúsculas
    - Reemplaza espacios por guiones bajos
    """
    df.columns = (df.columns
                  .str.strip()
                  .str.replace('"', '', regex=False)
                  .str.replace("'", '', regex=False)
                  .str.lower()
                  .str.replace(' ', '_')
                  .str.replace('-', '_'))
    return df

def detectar_separador(filepath, max_lines=5):
    """
    Detecta automáticamente el separador de un archivo CSV.
    """
    separadores = [';', ',', '\t', '|']
    
    with open(filepath, 'r', encoding='utf-8') as f:
        primeras_lineas = [f.readline() for _ in range(max_lines)]
    
    conteos = {sep: sum(linea.count(sep) for linea in primeras_lineas) 
               for sep in separadores}
    
    return max(conteos, key=conteos.get)

def info_basica_tabla(df, nombre):
    """
    Imprime información básica de una tabla.
    """
    print(f"\n{'='*70}")
    print(f"📋 TABLA: {nombre}")
    print(f"{'='*70}")
    print(f"   Dimensiones: {df.shape[0]:,} filas × {df.shape[1]} columnas")
    print(f"   Memoria: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    print(f"   Columnas: {', '.join(df.columns.tolist())}")

print("✅ Funciones auxiliares definidas")

✅ Funciones auxiliares definidas


### 1.3 Muestreo y Carga de Datos

In [12]:
def crear_muestras_y_cargar(input_dir, output_dir, sample_percentage, csv_files):
    """
    Lee archivos originales, crea muestras y carga los DataFrames.
    """
    print("\n" + "="*70)
    print("🔄 INICIANDO PROCESO DE MUESTREO Y CARGA")
    print("="*70)
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"✅ Directorio de salida creado: {output_dir}")
    
    resultados = []
    
    for filename in csv_files:
        input_filepath = os.path.join(input_dir, filename)
        output_filepath = os.path.join(output_dir, filename)
        table_name = filename.replace('.csv', '').upper()
        
        if not os.path.exists(input_filepath):
            print(f"⚠️  Archivo no encontrado: {filename}")
            continue
        
        try:
            # Detectar separador
            sep = detectar_separador(input_filepath)
            
            # Leer archivo original
            df = pd.read_csv(input_filepath, sep=sep, low_memory=False)
            df = limpiar_nombres_columnas(df)
            
            original_rows = len(df)
            
            # Aplicar muestreo
            if original_rows > 1000:
                sampled_df = df.sample(frac=sample_percentage, random_state=RANDOM_SEED)
            else:
                sampled_df = df.copy()
            
            sampled_rows = len(sampled_df)
            
            # Guardar muestra
            sampled_df.to_csv(output_filepath, index=False, sep=';')
            
            # Almacenar en memoria
            dataframes[table_name] = sampled_df
            
            # Registrar resultado
            resultados.append({
                'Tabla': table_name,
                'Filas Originales': original_rows,
                'Filas Muestra': sampled_rows,
                'Columnas': len(sampled_df.columns),
                '% Muestreado': (sampled_rows / original_rows * 100)
            })
            
            print(f"✅ {table_name:12s} | Original: {original_rows:>7,} | Muestra: {sampled_rows:>6,} filas")
            
        except Exception as e:
            print(f"❌ Error procesando {filename}: {str(e)}")
    
    # Resumen del muestreo
    print("\n" + "="*70)
    print("📊 RESUMEN DEL MUESTREO")
    print("="*70)
    df_resumen = pd.DataFrame(resultados)
    print(df_resumen.to_string(index=False))
    
    return dataframes

# Ejecutar muestreo y carga
dataframes = crear_muestras_y_cargar(INPUT_DIR, OUTPUT_DIR, SAMPLE_PERCENTAGE, CSV_FILES)


🔄 INICIANDO PROCESO DE MUESTREO Y CARGA
✅ ACCOUNT      | Original:   4,500 | Muestra:    225 filas
✅ CLIENT       | Original:   5,369 | Muestra:    268 filas
✅ DISP         | Original:   5,369 | Muestra:    268 filas
✅ LOAN         | Original:     682 | Muestra:    682 filas
✅ ORDER        | Original:   6,471 | Muestra:    324 filas
✅ TRANS        | Original: 1,056,320 | Muestra: 52,816 filas
✅ DISTRICT     | Original:      77 | Muestra:     77 filas
✅ CARD         | Original:     892 | Muestra:    892 filas

📊 RESUMEN DEL MUESTREO
   Tabla  Filas Originales  Filas Muestra  Columnas  % Muestreado
 ACCOUNT              4500            225         4          5.00
  CLIENT              5369            268         3          4.99
    DISP              5369            268         4          4.99
    LOAN               682            682         7        100.00
   ORDER              6471            324         6          5.01
   TRANS           1056320          52816        10          5.00

<a id='2-estructura'></a>
## 2. 📐 Análisis de Estructura y Esquema

### 2.1 Vista General del Dataset

In [14]:
print("\n" + "="*70)
print("🗂️  ESTRUCTURA GENERAL DEL DATASET BERKA")
print("="*70)

resumen_general = []

for nombre, df in dataframes.items():
    resumen_general.append({
        'Tabla': nombre,
        'Filas': len(df),
        'Columnas': len(df.columns),
        'Tamaño (KB)': df.memory_usage(deep=True).sum() / 1024,
        'Clave Primaria': PRIMARY_KEYS.get(nombre, 'N/A')
    })

df_resumen_general = pd.DataFrame(resumen_general)
df_resumen_general = df_resumen_general.sort_values('Filas', ascending=False)
print(df_resumen_general.to_string(index=False))

print(f"\n📊 Total de tablas: {len(dataframes)}")
print(f"📊 Total de filas (muestra): {df_resumen_general['Filas'].sum():,}")
print(f"📊 Tamaño total: {df_resumen_general['Tamaño (KB)'].sum():.2f} KB")



🗂️  ESTRUCTURA GENERAL DEL DATASET BERKA
   Tabla  Filas  Columnas  Tamaño (KB) Clave Primaria
   TRANS  52816        10     12554.89       trans_id
    CARD    892         4       118.20        card_id
    LOAN    682         7        65.40        loan_id
   ORDER    324         6        45.54       order_id
  CLIENT    268         3         8.38      client_id
    DISP    268         4        22.65        disp_id
 ACCOUNT    225         4        21.28     account_id
DISTRICT     77        16        24.34            N/A

📊 Total de tablas: 8
📊 Total de filas (muestra): 55,552
📊 Tamaño total: 12860.68 KB


### 2.2 Esquema Detallado por Tabla

In [15]:
def analizar_esquema_tabla(df, nombre):
    """
    Analiza el esquema de una tabla en detalle.
    """
    print(f"\n{'='*70}")
    print(f"📋 ESQUEMA: {nombre}")
    print(f"{'='*70}")
    
    esquema = pd.DataFrame({
        'Columna': df.columns,
        'Tipo': df.dtypes.values,
        'No Nulos': df.count().values,
        'Nulos': df.isnull().sum().values,
        '% Nulos': (df.isnull().sum() / len(df) * 100).values,
        'Únicos': [df[col].nunique() for col in df.columns],
        'Ejemplo': [df[col].iloc[0] if len(df) > 0 else None for col in df.columns]
    })
    
    print(esquema.to_string(index=False))
    
    # Identificar columnas problemáticas
    cols_muchos_nulos = esquema[esquema['% Nulos'] > 50]['Columna'].tolist()
    if cols_muchos_nulos:
        print(f"\n⚠️  Columnas con >50% nulos: {', '.join(cols_muchos_nulos)}")
    
    # Identificar posibles IDs
    cols_id = [col for col in df.columns if 'id' in col.lower()]
    if cols_id:
        print(f"\n🔑 Columnas ID encontradas: {', '.join(cols_id)}")

# Analizar esquema de cada tabla
for nombre, df in dataframes.items():
    analizar_esquema_tabla(df, nombre)



📋 ESQUEMA: ACCOUNT
    Columna   Tipo  No Nulos  Nulos  % Nulos  Únicos          Ejemplo
 account_id  int64       225      0     0.00     225             2243
district_id  int64       225      0     0.00      75               25
  frequency object       225      0     0.00       3 POPLATEK MESICNE
       date  int64       225      0     0.00     212           960124

🔑 Columnas ID encontradas: account_id, district_id

📋 ESQUEMA: CLIENT
     Columna  Tipo  No Nulos  Nulos  % Nulos  Únicos  Ejemplo
   client_id int64       268      0     0.00     268      431
birth_number int64       268      0     0.00     267   276001
 district_id int64       268      0     0.00      75       24

🔑 Columnas ID encontradas: client_id, district_id

📋 ESQUEMA: DISP
   Columna   Tipo  No Nulos  Nulos  % Nulos  Únicos Ejemplo
   disp_id  int64       268      0     0.00     268     431
 client_id  int64       268      0     0.00     268     431
account_id  int64       268      0     0.00     267     353
   

### 2.3 Detección de Tipos de Datos Incorrectos

In [16]:

print("\n" + "="*70)
print("🔍 DETECCIÓN DE TIPOS DE DATOS INCORRECTOS")
print("="*70)

# Columnas que deberían ser fechas (formato YYMMDD o YYYYMMDD)
COLUMNAS_FECHA = {
    'ACCOUNT': ['date'],
    'LOAN': ['date'],
    'TRANS': ['date'],
    'CARD': ['issued']
}

# Columnas que contienen fechas en birth_number
COLUMNAS_FECHA_ESPECIAL = {
    'CLIENT': ['birth_number']  # YYMMDD con lógica de género
}

def verificar_formato_fecha(df, col):
    """
    Verifica si una columna tiene formato de fecha incorrecto (número entero).
    """
    if col in df.columns:
        tipo_actual = df[col].dtype
        ejemplo = df[col].iloc[0] if len(df) > 0 else None
        
        if pd.api.types.is_numeric_dtype(df[col]):
            return True, tipo_actual, ejemplo
    
    return False, None, None

problemas_fecha = []

for tabla, columnas in COLUMNAS_FECHA.items():
    if tabla in dataframes:
        df = dataframes[tabla]
        for col in columnas:
            es_problema, tipo, ejemplo = verificar_formato_fecha(df, col)
            if es_problema:
                problemas_fecha.append({
                    'Tabla': tabla,
                    'Columna': col,
                    'Tipo Actual': tipo,
                    'Tipo Esperado': 'datetime',
                    'Ejemplo': ejemplo,
                    'Acción': 'Convertir de YYMMDD a YYYY-MM-DD'
                })

for tabla, columnas in COLUMNAS_FECHA_ESPECIAL.items():
    if tabla in dataframes:
        df = dataframes[tabla]
        for col in columnas:
            es_problema, tipo, ejemplo = verificar_formato_fecha(df, col)
            if es_problema:
                problemas_fecha.append({
                    'Tabla': tabla,
                    'Columna': col,
                    'Tipo Actual': tipo,
                    'Tipo Esperado': 'datetime + género',
                    'Ejemplo': ejemplo,
                    'Acción': 'Extraer fecha de nacimiento y género'
                })

if problemas_fecha:
    df_problemas = pd.DataFrame(problemas_fecha)
    print(df_problemas.to_string(index=False))
    print(f"\n❌ Se encontraron {len(problemas_fecha)} columnas con formato de fecha incorrecto")
else:
    print("✅ No se encontraron problemas con formatos de fecha")



🔍 DETECCIÓN DE TIPOS DE DATOS INCORRECTOS
  Tabla      Columna Tipo Actual     Tipo Esperado  Ejemplo                               Acción
ACCOUNT         date       int64          datetime   960124     Convertir de YYMMDD a YYYY-MM-DD
   LOAN         date       int64          datetime   930705     Convertir de YYMMDD a YYYY-MM-DD
  TRANS         date       int64          datetime   980320     Convertir de YYMMDD a YYYY-MM-DD
 CLIENT birth_number       int64 datetime + género   276001 Extraer fecha de nacimiento y género

❌ Se encontraron 4 columnas con formato de fecha incorrecto


<a id='3-calidad'></a>
## 3. 🔍 Análisis de Calidad de Datos

### 3.1 Análisis de Valores Nulos

In [17]:
def analizar_valores_nulos(df, nombre):
    """
    Analiza valores nulos y vacíos en detalle.
    """
    # Reemplazar valores vacíos por NaN
    df_temp = df.replace({'': np.nan, ' ': np.nan, 'NULL': np.nan, 'null': np.nan})
    
    nulos = df_temp.isnull().sum()
    total = len(df_temp)
    porcentaje = (nulos / total * 100)
    
    info_nulos = pd.DataFrame({
        'Columna': df_temp.columns,
        'Nulos': nulos.values,
        '% Nulos': porcentaje.values,
        'No Nulos': (total - nulos).values
    }).sort_values('% Nulos', ascending=False)
    
    # Filtrar solo columnas con nulos
    info_nulos = info_nulos[info_nulos['Nulos'] > 0]
    
    return info_nulos

print("\n" + "="*70)
print("❓ ANÁLISIS DE VALORES NULOS Y VACÍOS")
print("="*70)

resumen_nulos = []

for nombre, df in dataframes.items():
    info_nulos = analizar_valores_nulos(df, nombre)
    
    print(f"\n### 📋 {nombre} ({len(df):,} filas)")
    
    if not info_nulos.empty:
        print(info_nulos.to_string(index=False))
        
        # Interpretación de negocio
        if nombre == 'TRANS' and 'bank' in info_nulos['Columna'].values:
            print("\n💡 Interpretación: Los nulos en 'bank' y 'account' son ESPERADOS")
            print("   → Representan transacciones internas o en efectivo (VYBER, VKLAD)")
        
        if nombre == 'ORDER' and 'bank_to' in info_nulos['Columna'].values:
            print("\n💡 Interpretación: Los nulos en 'bank_to' y 'account_to' son ESPERADOS")
            print("   → Representan órdenes de pago internas")
        
        resumen_nulos.append({
            'Tabla': nombre,
            'Columnas con Nulos': len(info_nulos),
            'Max % Nulos': info_nulos['% Nulos'].max()
        })
    else:
        print("✅ No se encontraron valores nulos")
        resumen_nulos.append({
            'Tabla': nombre,
            'Columnas con Nulos': 0,
            'Max % Nulos': 0
        })

print("\n" + "="*70)
print("📊 RESUMEN DE NULOS")
print("="*70)
df_resumen_nulos = pd.DataFrame(resumen_nulos)
print(df_resumen_nulos.to_string(index=False))



❓ ANÁLISIS DE VALORES NULOS Y VACÍOS

### 📋 ACCOUNT (225 filas)
✅ No se encontraron valores nulos

### 📋 CLIENT (268 filas)
✅ No se encontraron valores nulos

### 📋 DISP (268 filas)
✅ No se encontraron valores nulos

### 📋 LOAN (682 filas)
✅ No se encontraron valores nulos

### 📋 ORDER (324 filas)
 Columna  Nulos  % Nulos  No Nulos
k_symbol     71    21.91       253

### 📋 TRANS (52,816 filas)
  Columna  Nulos  % Nulos  No Nulos
     bank  39284    74.38     13532
  account  38184    72.30     14632
 k_symbol  26657    50.47     26159
operation   9274    17.56     43542

💡 Interpretación: Los nulos en 'bank' y 'account' son ESPERADOS
   → Representan transacciones internas o en efectivo (VYBER, VKLAD)

### 📋 DISTRICT (77 filas)
✅ No se encontraron valores nulos

### 📋 CARD (892 filas)
✅ No se encontraron valores nulos

📊 RESUMEN DE NULOS
   Tabla  Columnas con Nulos  Max % Nulos
 ACCOUNT                   0         0.00
  CLIENT                   0         0.00
    DISP               


### 3.2 Análisis de Duplicados

In [18]:
def analizar_duplicados(df, nombre, pk_col):
    """
    Analiza duplicados en la clave primaria y filas completas.
    """
    resultados = {
        'tabla': nombre,
        'pk': pk_col,
        'total_filas': len(df),
        'pk_duplicadas': 0,
        'filas_completas_duplicadas': 0
    }
    
    # Verificar que la PK existe
    if pk_col not in df.columns:
        print(f"   ❌ ALERTA: Columna PK '{pk_col}' no encontrada")
        return resultados
    
    # 1. Duplicados en la Clave Primaria
    pk_duplicates = df[df.duplicated(subset=[pk_col], keep=False)]
    resultados['pk_duplicadas'] = len(pk_duplicates)
    
    # 2. Duplicados de filas completas
    full_duplicates = df[df.duplicated(keep=False)]
    resultados['filas_completas_duplicadas'] = len(full_duplicates)
    
    return resultados

print("\n" + "="*70)
print("👯 ANÁLISIS DE DUPLICADOS")
print("="*70)

resultados_duplicados = []

for nombre, df in dataframes.items():
    if nombre in PRIMARY_KEYS:
        pk_col = PRIMARY_KEYS[nombre]
        
        print(f"\n### 🔑 {nombre} (PK: {pk_col})")
        
        resultado = analizar_duplicados(df, nombre, pk_col)
        resultados_duplicados.append(resultado)
        
        # Mostrar resultados
        if resultado['pk_duplicadas'] > 0:
            print(f"   ❌ ERROR CRÍTICO: {resultado['pk_duplicadas']} filas con PK duplicada")
            print(f"   → La clave primaria DEBE ser única")
        else:
            print(f"   ✅ Clave Primaria única ({len(df):,} valores únicos)")
        
        if resultado['filas_completas_duplicadas'] > 0:
            pct_dup = (resultado['filas_completas_duplicadas'] / len(df)) * 100
            print(f"   ⚠️  {resultado['filas_completas_duplicadas']} filas completamente duplicadas ({pct_dup:.2f}%)")
        else:
            print("   ✅ No hay filas completamente duplicadas")

# Resumen de duplicados
print("\n" + "="*70)
print("📊 RESUMEN DE DUPLICADOS")
print("="*70)
df_dup = pd.DataFrame(resultados_duplicados)
print(df_dup[['tabla', 'pk', 'total_filas', 'pk_duplicadas', 'filas_completas_duplicadas']].to_string(index=False))



👯 ANÁLISIS DE DUPLICADOS

### 🔑 ACCOUNT (PK: account_id)
   ✅ Clave Primaria única (225 valores únicos)
   ✅ No hay filas completamente duplicadas

### 🔑 CLIENT (PK: client_id)
   ✅ Clave Primaria única (268 valores únicos)
   ✅ No hay filas completamente duplicadas

### 🔑 DISP (PK: disp_id)
   ✅ Clave Primaria única (268 valores únicos)
   ✅ No hay filas completamente duplicadas

### 🔑 LOAN (PK: loan_id)
   ✅ Clave Primaria única (682 valores únicos)
   ✅ No hay filas completamente duplicadas

### 🔑 ORDER (PK: order_id)
   ✅ Clave Primaria única (324 valores únicos)
   ✅ No hay filas completamente duplicadas

### 🔑 TRANS (PK: trans_id)
   ✅ Clave Primaria única (52,816 valores únicos)
   ✅ No hay filas completamente duplicadas

### 🔑 CARD (PK: card_id)
   ✅ Clave Primaria única (892 valores únicos)
   ✅ No hay filas completamente duplicadas

📊 RESUMEN DE DUPLICADOS
  tabla         pk  total_filas  pk_duplicadas  filas_completas_duplicadas
ACCOUNT account_id          225              

### 3.3 Análisis de Consistencia de Datos

In [19]:
print("\n" + "="*70)
print("🎯 ANÁLISIS DE CONSISTENCIA DE DATOS")
print("="*70)

# Verificar consistency en columnas categóricas clave
if 'ACCOUNT' in dataframes:
    print("\n### 📋 ACCOUNT - Frecuencia de Estados de Cuenta")
    freq_counts = dataframes['ACCOUNT']['frequency'].value_counts()
    print(freq_counts)
    valores_esperados = ['POPLATEK MESICNE', 'POPLATEK TYDNE', 'POPLATEK PO OBRATU']
    valores_invalidos = [v for v in freq_counts.index if v not in valores_esperados]
    if valores_invalidos:
        print(f"   ⚠️  Valores no esperados encontrados: {valores_invalidos}")
    else:
        print("   ✅ Todos los valores son válidos")

if 'TRANS' in dataframes:
    print("\n### 📋 TRANS - Tipos y Operaciones")
    print("\nTipos de transacción:")
    print(dataframes['TRANS']['type'].value_counts())
    print("\nOperaciones:")
    print(dataframes['TRANS']['operation'].value_counts())
    
    # Verificar que type solo tenga 'PRIJEM' o 'VYDAJ'
    tipos_validos = ['PRIJEM', 'VYDAJ', 'CREDIT', 'WITHDRAWAL']
    tipos_encontrados = dataframes['TRANS']['type'].unique()
    tipos_invalidos = [t for t in tipos_encontrados if t not in tipos_validos]
    if tipos_invalidos:
        print(f"   ⚠️  Tipos de transacción no válidos: {tipos_invalidos}")

if 'LOAN' in dataframes:
    print("\n### 📋 LOAN - Estados de Préstamos")
    status_counts = dataframes['LOAN']['status'].value_counts()
    print(status_counts)
    print("\n   Interpretación:")
    print("   • A: Contrato terminado - Sin problemas")
    print("   • B: Contrato activo - Sin problemas")
    print("   • C: Contrato terminado - Préstamo pagado correctamente")
    print("   • D: Contrato activo - Cliente con deuda (RIESGO)")



🎯 ANÁLISIS DE CONSISTENCIA DE DATOS

### 📋 ACCOUNT - Frecuencia de Estados de Cuenta
frequency
POPLATEK MESICNE      205
POPLATEK TYDNE         18
POPLATEK PO OBRATU      2
Name: count, dtype: int64
   ✅ Todos los valores son válidos

### 📋 TRANS - Tipos y Operaciones

Tipos de transacción:
type
VYDAJ     31656
PRIJEM    20340
VYBER       820
Name: count, dtype: int64

Operaciones:
operation
VYBER             21825
PREVOD NA UCET    10255
VKLAD              7789
PREVOD Z UCTU      3277
VYBER KARTOU        396
Name: count, dtype: int64
   ⚠️  Tipos de transacción no válidos: ['VYBER']

### 📋 LOAN - Estados de Préstamos
status
C    403
A    203
D     45
B     31
Name: count, dtype: int64

   Interpretación:
   • A: Contrato terminado - Sin problemas
   • B: Contrato activo - Sin problemas
   • C: Contrato terminado - Préstamo pagado correctamente
   • D: Contrato activo - Cliente con deuda (RIESGO)


<a id='4-integridad'></a>
## 4. 🔗 Análisis de Integridad Relacional

### 4.1 Verificación de Unicidad de Claves Primarias

In [20]:

print("\n" + "="*70)
print("🔑 VERIFICACIÓN DE UNICIDAD DE CLAVES PRIMARIAS")
print("="*70)

pk_results = []

for nombre, pk_col in PRIMARY_KEYS.items():
    if nombre in dataframes:
        df = dataframes[nombre]
        
        if pk_col in df.columns:
            total_rows = len(df)
            unique_rows = df[pk_col].nunique()
            duplicates = total_rows - unique_rows
            
            pk_results.append({
                'Tabla': nombre,
                'PK': pk_col,
                'Total Filas': total_rows,
                'Valores Únicos': unique_rows,
                'Duplicados': duplicates,
                'Unicidad': '✅ Única' if total_rows == unique_rows else '❌ Duplicada'
            })
            
            if total_rows == unique_rows:
                print(f"✅ {nombre:12s} | PK '{pk_col}' es ÚNICA ({total_rows:,} filas)")
            else:
                print(f"❌ {nombre:12s} | PK '{pk_col}' tiene {duplicates:,} duplicados")
        else:
            print(f"⚠️  {nombre:12s} | PK '{pk_col}' NO ENCONTRADA")

df_pk_results = pd.DataFrame(pk_results)
print("\n" + df_pk_results.to_string(index=False))




🔑 VERIFICACIÓN DE UNICIDAD DE CLAVES PRIMARIAS
✅ ACCOUNT      | PK 'account_id' es ÚNICA (225 filas)
✅ CLIENT       | PK 'client_id' es ÚNICA (268 filas)
✅ DISP         | PK 'disp_id' es ÚNICA (268 filas)
✅ LOAN         | PK 'loan_id' es ÚNICA (682 filas)
✅ ORDER        | PK 'order_id' es ÚNICA (324 filas)
✅ TRANS        | PK 'trans_id' es ÚNICA (52,816 filas)
✅ CARD         | PK 'card_id' es ÚNICA (892 filas)

  Tabla         PK  Total Filas  Valores Únicos  Duplicados Unicidad
ACCOUNT account_id          225             225           0  ✅ Única
 CLIENT  client_id          268             268           0  ✅ Única
   DISP    disp_id          268             268           0  ✅ Única
   LOAN    loan_id          682             682           0  ✅ Única
  ORDER   order_id          324             324           0  ✅ Única
  TRANS   trans_id        52816           52816           0  ✅ Única
   CARD    card_id          892             892           0  ✅ Única


### 4.2 Verificación de Integridad Referencial (Claves Foráneas)

In [21]:
print("\n" + "="*70)
print("🔗 VERIFICACIÓN DE INTEGRIDAD REFERENCIAL (FK → PK)")
print("="*70)

fk_results = []

for (fk_table, fk_col), (pk_table, pk_col) in FOREIGN_KEYS.items():
    if fk_table in dataframes and pk_table in dataframes:
        df_fk = dataframes[fk_table]
        df_pk = dataframes[pk_table]
        
        # Ajustar nombre de columna si es necesario
        pk_col_check = pk_col if pk_col in df_pk.columns else 'a1'
        
        if fk_col in df_fk.columns and pk_col_check in df_pk.columns:
            # Valores de FK que NO existen en PK
            fk_values = df_fk[fk_col].dropna().unique()
            pk_values = df_pk[pk_col_check].dropna().unique()
            
            missing_refs = len([v for v in fk_values if v not in pk_values])
            total_fk_values = len(fk_values)
            missing_pct = (missing_refs / total_fk_values * 100) if total_fk_values > 0 else 0
            
            fk_results.append({
                'FK Tabla': fk_table,
                'FK Columna': fk_col,
                'PK Tabla': pk_table,
                'PK Columna': pk_col,
                'FK Valores': total_fk_values,
                'Sin Referencia': missing_refs,
                '% Huérfanos': missing_pct,
                'Estado': '✅ OK' if missing_refs == 0 else '❌ ERROR'
            })
            
            if missing_refs == 0:
                print(f"✅ {fk_table}.{fk_col} → {pk_table}.{pk_col} | Integridad OK")
            else:
                print(f"❌ {fk_table}.{fk_col} → {pk_table}.{pk_col} | {missing_refs:,} valores sin referencia ({missing_pct:.1f}%)")
        else:
            print(f"⚠️  {fk_table}.{fk_col} → {pk_table}.{pk_col} | Columnas no encontradas")

if fk_results:
    df_fk_results = pd.DataFrame(fk_results)
    print("\n" + "="*70)
    print("📊 RESUMEN DE INTEGRIDAD REFERENCIAL")
    print("="*70)
    print(df_fk_results.to_string(index=False))
    
    # Análisis del problema de muestreo
    errores_criticos = df_fk_results[df_fk_results['% Huérfanos'] > 50]
    if not errores_criticos.empty:
        print("\n⚠️  ALERTA CRÍTICA: Problemas de integridad debido al muestreo independiente")
        print("   → Causa: Muestreo del 5% por tabla rompe las relaciones FK-PK")
        print("   → Impacto: En producción (dataset completo), estos errores NO deberían existir")
        print("   → Acción ETL: Validar integridad en datos completos, pero NO eliminar registros huérfanos")


🔗 VERIFICACIÓN DE INTEGRIDAD REFERENCIAL (FK → PK)
❌ DISP.account_id → ACCOUNT.account_id | 253 valores sin referencia (94.8%)
✅ DISP.client_id → CLIENT.client_id | Integridad OK
❌ LOAN.account_id → ACCOUNT.account_id | 648 valores sin referencia (95.0%)
❌ ORDER.account_id → ACCOUNT.account_id | 301 valores sin referencia (95.6%)
❌ TRANS.account_id → ACCOUNT.account_id | 4,239 valores sin referencia (95.1%)
❌ CARD.disp_id → DISP.disp_id | 844 valores sin referencia (94.6%)

📊 RESUMEN DE INTEGRIDAD REFERENCIAL
FK Tabla FK Columna PK Tabla PK Columna  FK Valores  Sin Referencia  % Huérfanos  Estado
    DISP account_id  ACCOUNT account_id         267             253        94.76 ❌ ERROR
    DISP  client_id   CLIENT  client_id         268               0         0.00    ✅ OK
    LOAN account_id  ACCOUNT account_id         682             648        95.01 ❌ ERROR
   ORDER account_id  ACCOUNT account_id         315             301        95.56 ❌ ERROR
   TRANS account_id  ACCOUNT account_id

<a id='5-descriptivo'></a>
## 5. 📊 Análisis Descriptivo por Tabla

### 5.1 Estadísticas Descriptivas de Variables Numéricas

In [22]:
print("\n" + "="*70)
print("📈 ESTADÍSTICAS DESCRIPTIVAS - VARIABLES NUMÉRICAS")
print("="*70)

# Definir columnas numéricas clave por tabla
NUMERIC_COLS_KEY = {
    'TRANS': ['amount', 'balance'],
    'LOAN': ['amount', 'payments', 'duration'],
    'ORDER': ['amount'],
    'ACCOUNT': [],  # Solo tiene IDs y fechas
    'CARD': [],
    'DEMOGRAPHIC': ['a4', 'a11', 'a12', 'a13', 'a15', 'a16']  # Población, salarios, desempleo, etc.
}

for tabla, columnas in NUMERIC_COLS_KEY.items():
    if tabla in dataframes and columnas:
        df = dataframes[tabla]
        
        # Filtrar columnas que realmente existen y son numéricas
        cols_disponibles = [col for col in columnas 
                           if col in df.columns and pd.api.types.is_numeric_dtype(df[col])]
        
        if cols_disponibles:
            print(f"\n### 💰 {tabla}")
            
            # Estadísticas con percentiles clave
            stats = df[cols_disponibles].describe(percentiles=[.01, .05, .10, .25, .50, .75, .90, .95, .99])
            print(stats)
            
            # Análisis adicional
            for col in cols_disponibles:
                print(f"\n   📊 {col}:")
                print(f"      • Rango: {df[col].min():,.2f} a {df[col].max():,.2f}")
                print(f"      • IQR (Q3-Q1): {df[col].quantile(0.75) - df[col].quantile(0.25):,.2f}")
                print(f"      • Coef. Variación: {(df[col].std() / df[col].mean()):.2%}")
                
                # Detectar asimetrías
                skew = df[col].skew()
                if abs(skew) > 1:
                    print(f"      • ⚠️  Distribución muy asimétrica (skew: {skew:.2f})")


📈 ESTADÍSTICAS DESCRIPTIVAS - VARIABLES NUMÉRICAS

### 💰 TRANS
        amount   balance
count 52816.00  52816.00
mean   5972.97  38530.29
std    9645.73  22059.96
min       0.20 -27966.70
1%       14.60   3449.76
5%       14.60  13636.68
10%      14.60  16289.50
25%     131.80  22463.88
50%    2066.00  33160.95
75%    6860.50  49586.80
90%   18641.50  68836.55
95%   26300.00  82551.62
99%   45701.30 108178.63
max   87300.00 209637.00

   📊 amount:
      • Rango: 0.20 a 87,300.00
      • IQR (Q3-Q1): 6,728.70
      • Coef. Variación: 161.49%
      • ⚠️  Distribución muy asimétrica (skew: 2.59)

   📊 balance:
      • Rango: -27,966.70 a 209,637.00
      • IQR (Q3-Q1): 27,122.93
      • Coef. Variación: 57.25%
      • ⚠️  Distribución muy asimétrica (skew: 1.23)

### 💰 LOAN
         amount  payments  duration
count    682.00    682.00    682.00
mean  151410.18   4190.66     36.49
std   113372.41   2215.83     17.08
min     4980.00    304.00     12.00
1%     12387.24    412.72     12.00
5

### 5.2 Análisis de Variables Categóricas

In [23]:
print("\n" + "="*70)
print("📊 ANÁLISIS DE VARIABLES CATEGÓRICAS")
print("="*70)

# Definir columnas categóricas clave
CATEGORICAL_COLS_KEY = {
    'ACCOUNT': ['frequency'],
    'TRANS': ['type', 'operation', 'k_symbol'],
    'LOAN': ['status'],
    'ORDER': ['k_symbol'],
    'CARD': ['type'],
    'DISP': ['type']
}

for tabla, columnas in CATEGORICAL_COLS_KEY.items():
    if tabla in dataframes:
        df = dataframes[tabla]
        
        print(f"\n### 📋 {tabla}")
        
        for col in columnas:
            if col in df.columns:
                print(f"\n   🔹 {col}:")
                value_counts = df[col].value_counts()
                value_pcts = df[col].value_counts(normalize=True) * 100
                
                resultado = pd.DataFrame({
                    'Valor': value_counts.index,
                    'Frecuencia': value_counts.values,
                    'Porcentaje': value_pcts.values
                })
                
                print(resultado.to_string(index=False))
                
                # Alertas
                if len(value_counts) == 1:
                    print("      ⚠️  Columna con un solo valor (considerar eliminar)")
                if len(value_counts) > 50:
                    print(f"      ⚠️  Alta cardinalidad ({len(value_counts)} valores únicos)")



📊 ANÁLISIS DE VARIABLES CATEGÓRICAS

### 📋 ACCOUNT

   🔹 frequency:
             Valor  Frecuencia  Porcentaje
  POPLATEK MESICNE         205       91.11
    POPLATEK TYDNE          18        8.00
POPLATEK PO OBRATU           2        0.89

### 📋 TRANS

   🔹 type:
 Valor  Frecuencia  Porcentaje
 VYDAJ       31656       59.94
PRIJEM       20340       38.51
 VYBER         820        1.55

   🔹 operation:
         Valor  Frecuencia  Porcentaje
         VYBER       21825       50.12
PREVOD NA UCET       10255       23.55
         VKLAD        7789       17.89
 PREVOD Z UCTU        3277        7.53
  VYBER KARTOU         396        0.91

   🔹 k_symbol:
      Valor  Frecuencia  Porcentaje
       UROK        9274       32.23
     SLUZBY        7894       27.43
       SIPO        5837       20.29
                   2615        9.09
     DUCHOD        1494        5.19
   POJISTNE         915        3.18
       UVER         657        2.28
SANKC. UROK          88        0.31

### 📋 LOAN

   🔹 s

### 5.3 Análisis de Distribución Temporal

In [24]:
print("\n" + "="*70)
print("📅 ANÁLISIS TEMPORAL (Basado en columnas de fecha como enteros)")
print("="*70)

# Nota: Las fechas están en formato YYMMDD como enteros
# Ejemplo: 930101 = 1 de enero de 1993

def parse_fecha_berka(fecha_int):
    """
    Convierte fecha en formato YYMMDD (int) a año.
    Asume siglo 19 para años >= 93.
    """
    try:
        fecha_str = str(int(fecha_int))
        yy = int(fecha_str[:2])
        year = 1900 + yy if yy >= 93 else 2000 + yy
        return year
    except:
        return None

# Analizar distribución de ACCOUNT por año de apertura
if 'ACCOUNT' in dataframes and 'date' in dataframes['ACCOUNT'].columns:
    print("\n### 📅 ACCOUNT - Distribución por Año de Apertura")
    df_account = dataframes['ACCOUNT'].copy()
    df_account['year'] = df_account['date'].apply(parse_fecha_berka)
    
    year_dist = df_account['year'].value_counts().sort_index()
    print(year_dist)
    
    print(f"\n   📊 Rango: {year_dist.index.min()} - {year_dist.index.max()}")
    print(f"   📊 Año con más aperturas: {year_dist.idxmax()} ({year_dist.max()} cuentas)")

# Analizar distribución de LOAN por año
if 'LOAN' in dataframes and 'date' in dataframes['LOAN'].columns:
    print("\n### 📅 LOAN - Distribución por Año de Préstamo")
    df_loan = dataframes['LOAN'].copy()
    df_loan['year'] = df_loan['date'].apply(parse_fecha_berka)
    
    year_dist = df_loan['year'].value_counts().sort_index()
    print(year_dist)
    
    print(f"\n   📊 Rango: {year_dist.index.min()} - {year_dist.index.max()}")



📅 ANÁLISIS TEMPORAL (Basado en columnas de fecha como enteros)

### 📅 ACCOUNT - Distribución por Año de Apertura
year
1993    52
1994    26
1995    30
1996    63
1997    54
Name: count, dtype: int64

   📊 Rango: 1993 - 1997
   📊 Año con más aperturas: 1996 (63 cuentas)

### 📅 LOAN - Distribución por Año de Préstamo
year
1993     20
1994    101
1995     90
1996    117
1997    196
1998    158
Name: count, dtype: int64

   📊 Rango: 1993 - 1998


<a id='6-negocio'></a>
## 6. 💼 Análisis de Negocio y Patrones

### 6.1 Análisis de Comportamiento Transaccional

In [25]:
print("\n" + "="*70)
print("💳 ANÁLISIS DE COMPORTAMIENTO TRANSACCIONAL")
print("="*70)

if 'TRANS' in dataframes:
    df_trans = dataframes['TRANS']
    
    print("\n### 📊 Distribución de Transacciones por Tipo")
    tipo_dist = df_trans.groupby('type').agg({
        'trans_id': 'count',
        'amount': ['mean', 'median', 'sum']
    }).round(2)
    print(tipo_dist)
    
    print("\n### 📊 Análisis de Balance")
    print(f"   • Balance promedio: {df_trans['balance'].mean():,.2f}")
    print(f"   • Balance mediano: {df_trans['balance'].median():,.2f}")
    print(f"   • Balance mínimo: {df_trans['balance'].min():,.2f}")
    print(f"   • Balance máximo: {df_trans['balance'].max():,.2f}")
    
    # Cuentas con balance negativo
    balances_negativos = df_trans[df_trans['balance'] < 0]
    if not balances_negativos.empty:
        pct_neg = (len(balances_negativos) / len(df_trans)) * 100
        print(f"\n   ⚠️  Transacciones con balance negativo: {len(balances_negativos):,} ({pct_neg:.2f}%)")
        print(f"   • Balance negativo promedio: {balances_negativos['balance'].mean():,.2f}")
        print("   → Indica existencia de sobregiros o líneas de crédito")
    
    print("\n### 📊 Top 5 Operaciones Más Comunes")
    top_operations = df_trans['operation'].value_counts().head(5)
    print(top_operations)


💳 ANÁLISIS DE COMPORTAMIENTO TRANSACCIONAL

### 📊 Distribución de Transacciones por Tipo
       trans_id   amount                      
          count     mean   median          sum
type                                          
PRIJEM    20340  8012.10  1500.00 162966063.30
VYBER       820 12565.39 12190.50  10303618.00
VYDAJ     31656  4491.99  2024.00 142198566.00

### 📊 Análisis de Balance
   • Balance promedio: 38,530.29
   • Balance mediano: 33,160.95
   • Balance mínimo: -27,966.70
   • Balance máximo: 209,637.00

   ⚠️  Transacciones con balance negativo: 164 (0.31%)
   • Balance negativo promedio: -3,523.18
   → Indica existencia de sobregiros o líneas de crédito

### 📊 Top 5 Operaciones Más Comunes
operation
VYBER             21825
PREVOD NA UCET    10255
VKLAD              7789
PREVOD Z UCTU      3277
VYBER KARTOU        396
Name: count, dtype: int64


### 6.2 Análisis de Riesgo Crediticio

In [26]:
print("\n" + "="*70)
print("⚠️  ANÁLISIS DE RIESGO CREDITICIO")
print("="*70)

if 'LOAN' in dataframes:
    df_loan = dataframes['LOAN']
    
    print("\n### 📊 Distribución de Estados de Préstamos")
    status_dist = df_loan['status'].value_counts()
    status_pct = df_loan['status'].value_counts(normalize=True) * 100
    
    status_summary = pd.DataFrame({
        'Estado': status_dist.index,
        'Cantidad': status_dist.values,
        'Porcentaje': status_pct.values
    })
    print(status_summary.to_string(index=False))
    
    # Calcular tasa de riesgo
    estados_riesgo = ['B', 'D']  # B = en curso sin problemas, D = en curso con deuda
    prestamos_riesgo = df_loan[df_loan['status'].isin(estados_riesgo)]
    tasa_riesgo = (len(prestamos_riesgo) / len(df_loan)) * 100
    
    print(f"\n   📊 Tasa de Préstamos en Riesgo (B o D): {tasa_riesgo:.2f}%")
    
    # Análisis por monto de préstamo
    print("\n### 💰 Análisis por Monto de Préstamo y Estado")
    loan_by_status = df_loan.groupby('status')['amount'].agg(['count', 'mean', 'median', 'sum']).round(2)
    print(loan_by_status)
    
    # Comparar duración de préstamos por estado
    print("\n### ⏱️  Duración de Préstamos por Estado")
    duration_by_status = df_loan.groupby('status')['duration'].agg(['mean', 'median', 'min', 'max']).round(2)
    print(duration_by_status)



⚠️  ANÁLISIS DE RIESGO CREDITICIO

### 📊 Distribución de Estados de Préstamos
Estado  Cantidad  Porcentaje
     C       403       59.09
     A       203       29.77
     D        45        6.60
     B        31        4.55

   📊 Tasa de Préstamos en Riesgo (B o D): 11.14%

### 💰 Análisis por Monto de Préstamo y Estado
        count      mean    median       sum
status                                     
A         203  91641.46  79632.00  18603216
B          31 140720.90  96396.00   4362348
C         403 171410.35 153504.00  69078372
D          45 249284.53 260400.00  11217804

### ⏱️  Duración de Préstamos por Estado
        mean  median  min  max
status                        
A      22.23   24.00   12   60
B      25.55   24.00   12   60
C      43.44   48.00   12   60
D      46.13   48.00   12   60


### 6.3 Análisis Demográfico

In [27]:
print("\n" + "="*70)
print("🗺️  ANÁLISIS DEMOGRÁFICO POR REGIÓN")
print("="*70)

if 'DEMOGRAPHIC' in dataframes and 'ACCOUNT' in dataframes:
    df_demo = dataframes['DEMOGRAPHIC']
    df_account = dataframes['ACCOUNT']
    
    # Merge de cuentas con demografía
    df_merged = pd.merge(
        df_account,
        df_demo,
        left_on='district_id',
        right_on='a1',
        how='inner'
    )
    
    print("\n### 📊 Top 10 Regiones por Número de Cuentas")
    region_counts = df_merged['a3'].value_counts().head(10)
    print(region_counts)
    
    print("\n### 💰 Salario Promedio por Frecuencia de Estado de Cuenta")
    if 'a11' in df_merged.columns and 'frequency' in df_merged.columns:
        salary_by_freq = df_merged.groupby('frequency')['a11'].agg(['mean', 'median', 'count']).round(2)
        print(salary_by_freq)
    
    print("\n### 📊 Análisis de Desempleo por Región (Top 10)")
    if 'a13' in df_demo.columns and 'a3' in df_demo.columns:
        unemployment = df_demo.nsmallest(10, 'a13')[['a3', 'a13']]
        print("Regiones con MENOR desempleo:")
        print(unemployment.to_string(index=False))
        
        unemployment_high = df_demo.nlargest(10, 'a13')[['a3', 'a13']]
        print("\nRegiones con MAYOR desempleo:")
        print(unemployment_high.to_string(index=False))


🗺️  ANÁLISIS DEMOGRÁFICO POR REGIÓN


### 6.4 Análisis de Relación Cliente-Cuenta

In [28]:
print("\n" + "="*70)
print("👥 ANÁLISIS DE RELACIÓN CLIENTE-CUENTA")
print("="*70)

if 'DISP' in dataframes:
    df_disp = dataframes['DISP']
    
    print("\n### 📊 Distribución de Tipos de Disposición")
    disp_type = df_disp['type'].value_counts()
    disp_pct = df_disp['type'].value_counts(normalize=True) * 100
    
    disp_summary = pd.DataFrame({
        'Tipo': disp_type.index,
        'Cantidad': disp_type.values,
        'Porcentaje': disp_pct.values
    })
    print(disp_summary.to_string(index=False))
    
    print("\n   💡 Interpretación:")
    print("   • OWNER: Cliente es el propietario principal de la cuenta")
    print("   • DISPONENT: Cliente es un autorizado/usuario adicional")
    
    # Cuentas por cliente
    print("\n### 📊 Número de Disposiciones por Cliente")
    disp_per_client = df_disp.groupby('client_id').size()
    print(f"   • Promedio de disposiciones por cliente: {disp_per_client.mean():.2f}")
    print(f"   • Mediana: {disp_per_client.median():.0f}")
    print(f"   • Máximo: {disp_per_client.max():.0f}")
    
    clientes_multiples_cuentas = disp_per_client[disp_per_client > 1]
    pct_multiples = (len(clientes_multiples_cuentas) / len(disp_per_client)) * 100
    print(f"\n   📊 Clientes con múltiples cuentas: {len(clientes_multiples_cuentas):,} ({pct_multiples:.2f}%)")



👥 ANÁLISIS DE RELACIÓN CLIENTE-CUENTA

### 📊 Distribución de Tipos de Disposición
     Tipo  Cantidad  Porcentaje
    OWNER       231       86.19
DISPONENT        37       13.81

   💡 Interpretación:
   • OWNER: Cliente es el propietario principal de la cuenta
   • DISPONENT: Cliente es un autorizado/usuario adicional

### 📊 Número de Disposiciones por Cliente
   • Promedio de disposiciones por cliente: 1.00
   • Mediana: 1
   • Máximo: 1

   📊 Clientes con múltiples cuentas: 0 (0.00%)


<a id='7-outliers'></a>
## 7. 📈 Análisis de Outliers y Anomalías

### 7.1 Detección de Outliers (Método IQR)

In [29]:
def detectar_outliers_iqr(df, col_name, threshold=1.5):
    """
    Detecta outliers usando el método IQR (Rango Intercuartílico).
    threshold: Factor de IQR (1.5 = outliers moderados, 3.0 = outliers extremos)
    """
    Q1 = df[col_name].quantile(0.25)
    Q3 = df[col_name].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR
    
    outliers = df[(df[col_name] < lower_bound) | (df[col_name] > upper_bound)]
    
    return outliers, upper_bound, lower_bound, IQR

print("\n" + "="*70)
print("📈 DETECCIÓN DE OUTLIERS Y ANOMALÍAS")
print("="*70)

# Columnas financieras para análisis de outliers
FINANCIAL_COLS_OUTLIERS = {
    'TRANS': ['amount', 'balance'],
    'LOAN': ['amount', 'payments'],
    'ORDER': ['amount'],
    'DEMOGRAPHIC': ['a11', 'a12', 'a13']  # Salarios y tasas
}

outlier_summary = []

for tabla, columnas in FINANCIAL_COLS_OUTLIERS.items():
    if tabla in dataframes:
        df = dataframes[tabla]
        
        print(f"\n### 💰 {tabla}")
        
        for col in columnas:
            if col in df.columns and pd.api.types.is_numeric_dtype(df[col]):
                outliers, upper, lower, iqr = detectar_outliers_iqr(df, col)
                
                outlier_pct = (len(outliers) / len(df)) * 100
                
                print(f"\n   📊 {col}:")
                print(f"      • Q1: {df[col].quantile(0.25):,.2f}")
                print(f"      • Q3: {df[col].quantile(0.75):,.2f}")
                print(f"      • IQR: {iqr:,.2f}")
                print(f"      • Límite Inferior: {lower:,.2f}")
                print(f"      • Límite Superior: {upper:,.2f}")
                
                if not outliers.empty:
                    print(f"      • ⚠️  Outliers encontrados: {len(outliers):,} ({outlier_pct:.2f}%)")
                    print(f"      • Valor mínimo outlier: {outliers[col].min():,.2f}")
                    print(f"      • Valor máximo outlier: {outliers[col].max():,.2f}")
                    
                    # Interpretación de negocio
                    if col in ['amount', 'payments']:
                        print("      → Acción: Investigar transacciones/préstamos grandes (fraude, VIP, errores)")
                    elif col == 'balance':
                        balances_muy_negativos = outliers[outliers[col] < 0]
                        if not balances_muy_negativos.empty:
                            print(f"      → {len(balances_muy_negativos)} cuentas con sobregiros severos")
                else:
                    print("      • ✅ No se detectaron outliers significativos")
                
                outlier_summary.append({
                    'Tabla': tabla,
                    'Columna': col,
                    'Total Registros': len(df),
                    'Outliers': len(outliers),
                    '% Outliers': outlier_pct,
                    'Límite Superior': upper,
                    'Max Valor': df[col].max()
                })

# Resumen de outliers
print("\n" + "="*70)
print("📊 RESUMEN DE OUTLIERS")
print("="*70)
df_outliers = pd.DataFrame(outlier_summary)
print(df_outliers.to_string(index=False))



📈 DETECCIÓN DE OUTLIERS Y ANOMALÍAS

### 💰 TRANS

   📊 amount:
      • Q1: 131.80
      • Q3: 6,860.50
      • IQR: 6,728.70
      • Límite Inferior: -9,961.25
      • Límite Superior: 16,953.55
      • ⚠️  Outliers encontrados: 6,036 (11.43%)
      • Valor mínimo outlier: 16,959.00
      • Valor máximo outlier: 87,300.00
      → Acción: Investigar transacciones/préstamos grandes (fraude, VIP, errores)

   📊 balance:
      • Q1: 22,463.88
      • Q3: 49,586.80
      • IQR: 27,122.93
      • Límite Inferior: -18,220.51
      • Límite Superior: 90,271.19
      • ⚠️  Outliers encontrados: 1,732 (3.28%)
      • Valor mínimo outlier: -27,966.70
      • Valor máximo outlier: 209,637.00
      → 3 cuentas con sobregiros severos

### 💰 LOAN

   📊 amount:
      • Q1: 66,732.00
      • Q3: 210,654.00
      • IQR: 143,922.00
      • Límite Inferior: -149,151.00
      • Límite Superior: 426,537.00
      • ⚠️  Outliers encontrados: 18 (2.64%)
      • Valor mínimo outlier: 428,784.00
      • Valor m

### 7.2 Análisis de Anomalías Específicas

In [30]:

print("\n" + "="*70)
print("🔍 ANÁLISIS DE ANOMALÍAS ESPECÍFICAS")
print("="*70)

# 1. Transacciones con montos = 0
if 'TRANS' in dataframes:
    df_trans = dataframes['TRANS']
    trans_zero = df_trans[df_trans['amount'] == 0]
    if not trans_zero.empty:
        print(f"\n⚠️  Transacciones con monto = 0: {len(trans_zero):,}")
        print("   → Revisar: ¿Son transacciones de ajuste o errores?")

# 2. Préstamos con pagos inconsistentes
if 'LOAN' in dataframes:
    df_loan = dataframes['LOAN']
    # Calcular pago mensual esperado
    df_loan['expected_payment'] = df_loan['amount'] / df_loan['duration']
    df_loan['payment_diff'] = abs(df_loan['payments'] - df_loan['expected_payment'])
    df_loan['payment_diff_pct'] = (df_loan['payment_diff'] / df_loan['expected_payment']) * 100
    
    pagos_inconsistentes = df_loan[df_loan['payment_diff_pct'] > 10]  # >10% de diferencia
    if not pagos_inconsistentes.empty:
        print(f"\n⚠️  Préstamos con pagos inconsistentes: {len(pagos_inconsistentes):,}")
        print(f"   → Diferencia >10% entre pago mensual y pago esperado")
        print(f"   → Podría indicar cambios en términos o errores de cálculo")

# 3. Cuentas sin transacciones
if 'ACCOUNT' in dataframes and 'TRANS' in dataframes:
    accounts_with_trans = dataframes['TRANS']['account_id'].unique()
    all_accounts = dataframes['ACCOUNT']['account_id'].unique()
    accounts_no_trans = [acc for acc in all_accounts if acc not in accounts_with_trans]
    
    if accounts_no_trans:
        print(f"\n⚠️  Cuentas sin transacciones en la muestra: {len(accounts_no_trans)}")
        print("   → Nota: Esto puede ser debido al muestreo del 5%")

# 4. Clientes sin cuentas (por problema de FK)
if 'CLIENT' in dataframes and 'DISP' in dataframes:
    clients_with_accounts = dataframes['DISP']['client_id'].unique()
    all_clients = dataframes['CLIENT']['client_id'].unique()
    clients_no_accounts = [cli for cli in all_clients if cli not in clients_with_accounts]
    
    if clients_no_accounts:
        pct = (len(clients_no_accounts) / len(all_clients)) * 100
        print(f"\n⚠️  Clientes sin cuentas: {len(clients_no_accounts)} ({pct:.2f}%)")
        print("   → Nota: Esto puede ser debido al muestreo del 5%")


🔍 ANÁLISIS DE ANOMALÍAS ESPECÍFICAS

⚠️  Cuentas sin transacciones en la muestra: 5
   → Nota: Esto puede ser debido al muestreo del 5%


<a id='8-conclusiones'></a>
## 8. 📋 Conclusiones y Plan de Transformación ETL

### 8.1 Resumen Ejecutivo de Hallazgos

### 🎯 HALLAZGOS CRÍTICOS

#### 1. ❌ PROBLEMAS DE FORMATO Y ESTRUCTURA
   - Fechas almacenadas como enteros (YYMMDD) en todas las tablas
   - Cabeceras con comillas dobles y separador ';' en archivos originales
   - Columna 'a1' en DEMOGRAPHIC debería llamarse 'district_id'
   - Nombres de columnas inconsistentes (mezcla de códigos y nombres)

#### 2. ⚠️  PROBLEMAS DE CALIDAD DE DATOS
   - Valores nulos funcionales en TRANS.bank y ORDER.bank_to (representan transacciones internas)
   - No se encontraron duplicados en claves primarias (✅)
   - Balances negativos válidos (indican sobregiros o líneas de crédito)
   - Outliers significativos en TRANS.amount y LOAN.amount (requieren investigación)

#### 3. 🔗 PROBLEMAS DE INTEGRIDAD REFERENCIAL
   - 94-95% de FKs sin referencias en la muestra (causado por muestreo independiente)
   - En el dataset completo, esta tasa debería ser cercana a 0%
   - Acción: NO eliminar huérfanos en ETL, validar con dataset completo

#### 4. 💡 OPORTUNIDADES DE FEATURE ENGINEERING
   - Extraer edad y género desde CLIENT.birth_number
   - Calcular antigüedad de cuenta desde ACCOUNT.date
   - Crear variable target 'is_risky' en LOAN (status B o D)
   - Calcular fecha de finalización de préstamo en LOAN
   - Agregar transacciones por cuenta para análisis de comportamiento

#### 5. 📊 PATRONES DE NEGOCIO IDENTIFICADOS
   - ~95% de préstamos tienen estado A o C (bajo riesgo)
   - Existencia de balances negativos (sobregiros permitidos)
   - Mayoría de clientes tienen 1 sola cuenta
   - Concentración de cuentas en ciertas regiones geográficas



### 8.2 Plan de Transformación ETL Detallado

### 🔄 FASE 1: RAW → PROCESSED (Limpieza y Estandarización)

| ID | Problema | Acción de Transformación | Prioridad |
|----|----------|-------------------------|-----------|
| T1 | Separador y comillas en CSV | Configurar lectura con sep=';' y quitar comillas | 🔴 CRÍTICA |
| T2 | Nombres de columnas sucios | Aplicar limpieza: minúsculas, sin comillas, snake_case | 🔴 CRÍTICA |
| T3 | Fechas como enteros (YYMMDD) | Parsear a formato datetime YYYY-MM-DD | 🔴 CRÍTICA |
| T4 | Columna 'a1' en DEMOGRAPHIC | Renombrar a 'district_id' | 🟡 ALTA |
| T5 | Columnas 'a11', 'a12', etc. | Renombrar a nombres descriptivos (avg_salary, unemployment_rate) | 🟡 ALTA |
| T6 | Nulos funcionales en TRANS/ORDER | Reemplazar por 'INTERNAL' o 'NA_CASH' | 🟡 ALTA |
| T7 | Validar integridad referencial | Marcar registros huérfanos con flag 'is_orphan' | 🟢 MEDIA |
| T8 | Valores vacíos vs NULL | Estandarizar todos a NULL | 🟢 MEDIA |

### 🎯 FASE 2: PROCESSED → CURATED (Feature Engineering y Modelado)

| ID | Feature | Descripción | Tabla | Prioridad |
|----|---------|-------------|-------|-----------|
| F1 | age | Calcular edad desde birth_number | CLIENT | 🔴 CRÍTICA |
| F2 | gender | Extraer género desde birth_number (+50 en día = mujer) | CLIENT | 🔴 CRÍTICA |
| F3 | account_age_months | Antigüedad de cuenta desde fecha apertura | ACCOUNT | 🟡 ALTA |
| F4 | loan_end_date | Fecha fin = fecha inicio + duración | LOAN | 🟡 ALTA |
| F5 | is_risky | Target binario: 1 si status=B o D, 0 si A o C | LOAN | 🔴 CRÍTICA |
| F6 | is_trans_outlier | Flag para transacciones atípicas (IQR method) | TRANS | 🟢 MEDIA |
| F7 | is_loan_outlier | Flag para préstamos atípicos (>95 percentil) | LOAN | 🟢 MEDIA |
| F8 | avg_balance | Balance promedio por cuenta | ACCOUNT | 🟡 ALTA |
| F9 | total_transactions | Número de transacciones por cuenta | ACCOUNT | 🟡 ALTA |
| F10 | negative_balance_flag | 1 si alguna vez tuvo balance negativo | ACCOUNT | 🟢 MEDIA |

### 📊 FASE 3: MODELADO DIMENSIONAL (Para Análisis)

**Tablas de Hechos (Fact Tables):**
- fact_transactions: Agregación de TRANS por cuenta
- fact_loans: Tabla de préstamos enriquecida con features

**Tablas de Dimensiones (Dimension Tables):**
- dim_client: Clientes con edad, género, región
- dim_account: Cuentas con métricas agregadas
- dim_geographic: Datos demográficos por distrito
- dim_date: Dimensión de tiempo para análisis temporal


### 8.3 Recomendaciones Finales

### 🎯 RECOMENDACIONES PARA EL PIPELINE ETL

#### 1. 🔴 PRIORIDAD CRÍTICA - Implementar Inmediatamente
   - ✓ Configurar lectura correcta de CSVs (separador, encoding)
   - ✓ Parsear fechas de YYMMDD a datetime
   - ✓ Extraer edad y género desde birth_number
   - ✓ Crear variable target 'is_risky' para modelos de ML

#### 2. 🟡 PRIORIDAD ALTA - Implementar en Sprint 1
   - ✓ Renombrar columnas crípticas (a1 → district_id, a11 → avg_salary)
   - ✓ Tratar nulos funcionales con etiquetas de negocio
   - ✓ Calcular features temporales (antigüedad, fecha_fin_prestamo)
   - ✓ Validar integridad referencial en dataset completo

#### 3. 🟢 PRIORIDAD MEDIA - Implementar en Sprint 2
   - ✓ Marcar outliers para análisis de fraude
   - ✓ Agregar métricas por cuenta (balance promedio, total transacciones)
   - ✓ Crear flags de comportamiento (sobregiros, inactividad)
   - ✓ Implementar data quality checks automatizados

#### 4. 📊 ANÁLISIS ADICIONALES RECOMENDADOS
   - ✓ Análisis de serie temporal de transacciones
   - ✓ Segmentación de clientes por comportamiento
   - ✓ Modelo de predicción de riesgo crediticio
   - ✓ Análisis de correlación entre variables demográficas y riesgo

#### 5. ⚠️  ADVERTENCIAS IMPORTANTES
   - Los problemas de FK en la muestra NO deben replicarse en datos completos
   - NO eliminar registros huérfanos sin validar con datos completos
   - Los outliers pueden ser válidos (VIP, fraude) - marcar, no eliminar
   - Los balances negativos son válidos - indican sobregiros permitidos



In [3]:
import pandas as pd
import os

# --- CONFIGURACIÓN ---
# La ruta de tu archivo trans.csv original
INPUT_FILE = './data_original/trans_millon.csv'
# La ruta donde guardarás el archivo muestreado
OUTPUT_FILE = './data_original/trans.csv' # Usaremos el mismo nombre para simplificar la corrida de Glue
# Fracción de datos a mantener (10% de 1 millón es 100,000 filas)
SAMPLE_FRACTION = 0.5

def sample_transactions(input_path: str, output_path: str, frac: float):
    """Carga trans.csv, toma una muestra y la guarda, manteniendo la cabecera original."""
    
    print(f"Cargando el archivo: {input_path}")
    
    # Cargar solo las columnas necesarias para acelerar (opcional)
    # y asegurarse de que el separador sea el punto y coma
    try:
        # Nota: Usamos engine='python' si hay problemas con el separador o el tamaño
        df = pd.read_csv(input_path, sep=';', encoding='iso-8859-1')
    except FileNotFoundError:
        print(f"ERROR: No se encontró el archivo en la ruta: {input_path}")
        return

    total_rows = len(df)
    sample_size = int(total_rows * frac)
    
    print(f"Filas totales originales: {total_rows:,}")
    print(f"Tomando una muestra del {frac*100}% ({sample_size:,} filas)")

    # Tomar la muestra aleatoria
    df_sampled = df.sample(n=sample_size, random_state=42) # Usamos random_state para reproducibilidad
    
    # Guardar el archivo muestreado
    # Importante: Mantener el separador original (;) y no incluir el índice de Pandas
    df_sampled.to_csv(output_path, sep=';', index=False, header=True)
    
    print(f"\n✅ Archivo muestreado guardado exitosamente.")
    print(f"El nuevo archivo '{output_path}' contiene {len(df_sampled):,} filas.")

if __name__ == "__main__":
    # Asegúrate de colocar el archivo 'trans.csv' original en el mismo directorio
    # donde ejecutas este script, o ajusta la ruta en INPUT_FILE.
    sample_transactions(INPUT_FILE, OUTPUT_FILE, SAMPLE_FRACTION)

Cargando el archivo: ./data_original/trans_millon.csv


/tmp/ipykernel_9836/2609291857.py:21: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_path, sep=';', encoding='iso-8859-1')


Filas totales originales: 1,056,320
Tomando una muestra del 50.0% (528,160 filas)

✅ Archivo muestreado guardado exitosamente.
El nuevo archivo './data_original/trans.csv' contiene 528,160 filas.
